In [13]:
%load_ext autoreload
%autoreload 2
from final_algo import partial_cov, get_edges, adj, remove_adj_i
import numpy as np
import itertools
from paper_sims_util import their_random_graph_2, MCC, confusion, grid_graph
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt

In [120]:
p = 100
omega = their_random_graph_2(p, 0.05)#grid_graph(10)
sigma = np.linalg.inv(omega)
N = 10000
X = np.random.multivariate_normal(mean = np.zeros(p), cov = np.linalg.inv(omega), size = N)

In [121]:
N, p = X.shape
K = math.ceil(6 * np.log2(p))
batches = np.array_split(X, K)

In [122]:
l = -1
hypothesis_graph = np.ones((p,p))

valid_edge_exists = True

while valid_edge_exists:
    valid_edge_exists = False
    l = l+1
    print("Working on l = {}".format(l))
    for edge in get_edges(hypothesis_graph):
        i, j = edge
        #print('edge', i, j)
        adj_i = adj(hypothesis_graph, i)
        if len(adj_i) >= l+1:
            adj_i.remove(j)
            valid_edge_exists = True
        else:
            continue #to next iteration
        
        #print('after if-else')
        stop = False
        combos = list(itertools.combinations(adj_i, l))
        rhos = []
        for S in combos:
            S = list(S)
            all_K = remove_adj_i(p, S, i, j)
            for k in all_K:
                if not stop:
                    subset = sorted(S + [k] + [i] + [j])
                    d = np.random.randint(K)
                    data = batches[d]
                    sample_cov = np.cov(data.T)
                    rho = partial_cov(sample_cov, subset, i, j)
                    rhos.append(rho)
                    if rho < 0:
                        #print("deleted", i, j)
                        if omega[i,j] != 0:
                            pass
                            #print("BAD")
                            #print(rhos)
                        hypothesis_graph[i,j] = 0
                        hypothesis_graph[j, i] = 0
                        stop = True

Working on l = 0
Working on l = 1
Working on l = 2
Working on l = 3
Working on l = 4


In [123]:
confusion(hypothesis_graph, omega)

(48, 4888, 0, 14)

In [104]:
MCC(hypothesis_graph, omega)

0.8914933596431405

In [ ]:
TP, TN, FP, FN